In [19]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.logging.set_verbosity(tf.logging.ERROR)
mnist=input_data.read_data_sets("E:/ML_Dataset/MNIST/",one_hot=True)
tf.set_random_seed(1)

#hyprtparameters
lr=0.01
training_iters=100000
batch_size=128
n_inputs=28
n_steps=28
n_hidden_units=128
n_classes=10

Extracting E:/ML_Dataset/MNIST/train-images-idx3-ubyte.gz
Extracting E:/ML_Dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting E:/ML_Dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting E:/ML_Dataset/MNIST/t10k-labels-idx1-ubyte.gz


In [2]:
x=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.float32,[None,n_classes])

weights={
    #shape(28,128)
    'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    #shape(128,10)
    'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}

biases={
    #shape(128,)
    'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    #shape(10,)
    'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [14]:
def RNN(X,weights,biases):
    #X==>(128 batches*28steps,28 inputs)
    X=tf.reshape(X,[-1,n_inputs])
    
    X_in=tf.matmul(X,weights['in'])+biases['in']
    
    X_in=tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    
    with tf.variable_scope('scope1'):    
        lstm_cell=tf.contrib.rnn.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
    init_state=lstm_cell.zero_state(batch_size,dtype=tf.float32)
    
    with tf.variable_scope('scope2',reuse=tf.AUTO_REUSE):
        outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=init_state,time_major=False)
    
    results=tf.matmul(final_state[1],weights['out'])+biases['out']
    
    return results

In [16]:
pred=RNN(x,weights,biases)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
train_op=tf.train.AdamOptimizer(lr).minimize(cost)

### 训练RNN

In [21]:
correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step=0
    while step*batch_size<training_iters:
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape([batch_size,n_steps,n_inputs])
        sess.run([train_op],feed_dict={
            x:batch_xs,
            y:batch_ys,
        })
        if step%20==0:
            print(sess.run(accuracy,feed_dict={
                x:batch_xs,
                y:batch_ys
            }))
        step+=1

0.2265625
0.5625
0.71875
0.78125
0.875
0.8125
0.8046875
0.859375
0.8359375
0.8671875
0.828125
0.8671875
0.859375
0.90625
0.9140625
0.9296875
0.875
0.9296875
0.890625
0.9375
0.90625
0.96875
0.875
0.96875
0.953125
0.921875
0.953125
0.9375
0.9140625
0.9375
0.9375
0.953125
0.9375
0.9453125
0.9453125
0.953125
0.9453125
0.953125
0.9609375
0.9296875
